In [24]:
# -*- encoding: utf8-*-
import requests
import time
import random
import json
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#print(json.dumps(return_json, indent=2))

In [25]:
def get_top_songs():
    top_songs = []

    headers = {'Referer': 'https://kma.kkbox.com/charts/yearly/newrelease?lang=tc&terr=tw',
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55',
                'Content-Type': 'application/json'}
    url = 'https://kma.kkbox.com/charts/api/v1/yearly?category=297&lang=tc&limit=100&terr=tw&type=newrelease&year=2021'
    
    return_data = requests.get(url, headers=headers)
    return_json = json.loads(return_data.text)

    for song in return_json['data']['charts']['newrelease']:
        top_songs.append(re.split(' -| \(', song['song_name'])[0])
    
    return top_songs

In [26]:
# 獲取樂譜頁面
def get_pu(song_name):

    url = 'https://www.91pu.com.tw/plus/search.php?keyword=' + song_name + '&type='
    print(url)
    return_html = requests.get(url)
    soup = BeautifulSoup(return_html.text, 'html.parser')
    pu_url = 'https://www.91pu.com.tw' + soup.select('tr > td > a')[0].get('href')

    return pu_url

In [27]:
def get_chords_and_lyrics(pu_url):
    #chord, lyric = [], []
    tot = []
    options = Options()
    options.add_argument("--disable-notifications")
    options.headless = True

    driver = webdriver.Edge(options=options)
    driver.implicitly_wait(3)
    driver.get(pu_url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    tone = soup.find(id='tone_z').find_all('p')
    # print(tone)
    clean = re.compile('<.*?>')
    nbsp = re.compile(u'\xa0+')
    em = re.compile('\u2003')
    for tags in tone:
        output = tags.text
        output = re.sub(clean, '', output)
        output = re.sub(nbsp, ' ', output)
        output = re.sub(em, ' ', output)
        # print(output)
        
        # classes = tags.get('class')
        # if 'music' in classes:
        #     chord.append(output)
        # elif 'lyric' in classes:
        #     lyric.append(output)
        tot.append(output)
    driver.quit()
    return tot
    # return chord, lyric

In [28]:
top_songs = get_top_songs()

chords = []
lyrics = []
tots = []
for i in range(100):
    try:
        pu_url = get_pu(top_songs[i])
        #chord, lyric = get_chords_and_lyrics(pu_url)
        tot = get_chords_and_lyrics(pu_url)
        #chords.append(chord)
        #lyrics.append(lyric)
        tots.append(tot)
    except:
        tots.append(['no data'])
        #chords.append(['no chords'])
        #lyrics.append(['no lyrics'])



https://www.91pu.com.tw/plus/search.php?keyword=愛情你比我想的閣較偉大&type=
https://www.91pu.com.tw/plus/search.php?keyword=在這座城市遺失了你&type=
https://www.91pu.com.tw/plus/search.php?keyword=我很好騙&type=
https://www.91pu.com.tw/plus/search.php?keyword=四季予你&type=
https://www.91pu.com.tw/plus/search.php?keyword=阿拉斯加海灣&type=
https://www.91pu.com.tw/plus/search.php?keyword=因為你 所以我&type=
https://www.91pu.com.tw/plus/search.php?keyword=白月光與朱砂痣&type=
https://www.91pu.com.tw/plus/search.php?keyword=失重前幸褔&type=
https://www.91pu.com.tw/plus/search.php?keyword=如果能幸福&type=
https://www.91pu.com.tw/plus/search.php?keyword=星辰大海&type=
https://www.91pu.com.tw/plus/search.php?keyword=如果可以&type=
https://www.91pu.com.tw/plus/search.php?keyword=唯一&type=
https://www.91pu.com.tw/plus/search.php?keyword=清空&type=
https://www.91pu.com.tw/plus/search.php?keyword=漸冷&type=
https://www.91pu.com.tw/plus/search.php?keyword=緩緩&type=
https://www.91pu.com.tw/plus/search.php?keyword=愛我的時候&type=
https://www.91pu.com.tw/plus/search.php?k

In [32]:
f = open('total.txt', "w", encoding='UTF-8')
num = 0
for i in tots:
    f.write(top_songs[num]+"\n")
    for j in i:
        f.write(j + '\n')
    f.write("\n")
    num += 1
f.close()

In [30]:
# song_name='失重前幸福'

# pu_url = get_pu(song_name)
# chord, lyric = get_chords_and_lyrics(pu_url)
# print(chord)
# print(lyric)

In [31]:

# f = open('out.txt', "a", encoding='UTF-8')
# f.write(song_name+"\n")
# for j in chord:
#     f.write(j + ' ')
# f.write("\n")
# f.close()